## 1. 数据集下载
+ Mini-ImageNet：https://drive.google.com/drive/folders/17a09kkqVivZQFggCw9I_YboJ23tcexNM

+ 下载的数据集根目录dataset/下有train、val、test文件夹
+ 但由于该数据集用于元学习和小样本学习，数据集的划分不是从每个类别中进行采样的，所以需要进一步对下载的数据集重新进行train、val、test的划分，使train、val、test都包含全部的分类

In [2]:
import os, random, shutil

# 先把上述链接中下载好的val、test目录下的所有文件夹都剪切至train目录，完成合并
# 如下代码是使train、val、test都含有全部100个分类，并针对每个分类按照6:2:2的比例划分

def moveFile(fileDir, tarDir, rate):
    pathDir = os.listdir(fileDir)  # 取图片的原始路径
    filenumber = len(pathDir)
    if filenumber > 360: # 重新划分好的train目录下每个分类的图片数量=360，此行是为了防止重复进行划分
        rate = rate  # 自定义抽取图片的比例，比方说100张抽10张，那就是0.1
        picknumber = int(filenumber * rate)  # 按照rate比例从文件夹中取一定数量图片
        sample = random.sample(pathDir, picknumber)  # 随机选取picknumber数量的样本图片
        print(sample)
        print(len(sample))
        for name in sample:
            shutil.move(fileDir + '/' + name, tarDir + '/' + name)

data_dir = "dataset/train"

for classes_num in os.listdir(data_dir):
    src_dir = os.path.join("./dataset/train/",classes_num)
    dst_dir_val = os.path.join("./dataset/val/", classes_num)
    dst_dir_test = os.path.join("./dataset/test/", classes_num)
    
    if not os.path.exists(dst_dir_val):
        os.mkdir(dst_dir_val)
    
    if not os.path.exists(dst_dir_test):
        os.mkdir(dst_dir_test)
    
    moveFile(src_dir, dst_dir_val, 0.2)
    moveFile(src_dir, dst_dir_test, 0.25)


+ 重新划分后，train、val、test每个文件夹下目录结构如下：
```
.
└── train/val/test 
     ├── class1
     │    ├── 000000000001.jpg
     │    ├── 000000000002.jpg
     │    ├── ...
     ├── class2
     │    ├── 000000000001.jpg
     │    ├── 000000000002.jpg
     │    ├── ...
     ├── class3
     │    ├── 000000000001.jpg
     │    ├── 000000000002.jpg
     │    ├── ...
     ├── class100
          ├── 000000000001.jpg
          ├── 000000000002.jpg
          ├── ...
```

## 2. 加载数据集

In [3]:
import os
import math
import numpy as np
from PIL import Image, ImageFile
from mindspore import dtype as mstype
import mindspore.dataset as de
import mindspore.dataset.vision.c_transforms as vision_C
import mindspore.dataset.transforms.c_transforms as normal_C

ImageFile.LOAD_TRUNCATED_IMAGES = True


def classification_dataset(data_dir, image_size, per_batch_size, max_epoch, rank, group_size,
                           mode='train',
                           input_mode='folder',
                           root='',
                           num_parallel_workers=None,
                           shuffle=None,
                           sampler=None,
                           class_indexing=None,
                           drop_remainder=True,
                           transform=None,
                           target_transform=None):
   

    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255]
    std = [0.229 * 255, 0.224 * 255, 0.225 * 255]

    if transform is None:
        if mode == 'train':
            transform_img = [
                vision_C.RandomCropDecodeResize(image_size, scale=(0.08, 1.0), ratio=(0.75, 1.333)),
                vision_C.RandomHorizontalFlip(prob=0.5),
                vision_C.RandomColorAdjust(brightness=0.4, contrast=0.4, saturation=0.4),
                vision_C.Normalize(mean=mean, std=std),
                vision_C.HWC2CHW()
            ]
        else:
            transform_img = [
                vision_C.Decode(),
                vision_C.Resize((256, 256)),
                vision_C.CenterCrop(image_size),
                vision_C.Normalize(mean=mean, std=std),
                vision_C.HWC2CHW()
            ]
    else:
        transform_img = transform

    if target_transform is None:
        transform_label = [
            normal_C.TypeCast(mstype.int32)
        ]
    else:
        transform_label = target_transform


    de_dataset = de.ImageFolderDataset(data_dir, num_parallel_workers=num_parallel_workers,
                                           shuffle=shuffle, sampler=sampler, class_indexing=class_indexing,
                                           num_shards=group_size, shard_id=rank)

    de_dataset = de_dataset.map(input_columns="image", num_parallel_workers=8, operations=transform_img)
    de_dataset = de_dataset.map(input_columns="label", num_parallel_workers=8, operations=transform_label)

    columns_to_project = ["image", "label"]
    de_dataset = de_dataset.project(columns=columns_to_project)

    de_dataset = de_dataset.batch(per_batch_size, drop_remainder=drop_remainder)
    de_dataset = de_dataset.repeat(1)

    return de_dataset

## 3. 构建Densenet121网络模型

In [4]:
import math
from collections import OrderedDict

import mindspore.nn as nn
import mindspore as ms
from mindspore.ops import operations as P
from mindspore.common import initializer as init
from utils.network_utils import default_recurisive_init, KaimingNormal


__all__ = ["DenseNet121", "DenseNet100"]

### 3.1 构建DenseBlock

In [5]:
class GlobalAvgPooling(nn.Cell):
    """
    GlobalAvgPooling function.
    """
    def __init__(self):
        super(GlobalAvgPooling, self).__init__()
        self.mean = P.ReduceMean(True)
        self.shape = P.Shape()
        self.reshape = P.Reshape()

    def construct(self, x):
        x = self.mean(x, (2, 3))
        b, c, _, _ = self.shape(x)
        x = self.reshape(x, (b, c))
        return x

class CommonHead(nn.Cell):
    def __init__(self, num_classes, out_channels):
        super(CommonHead, self).__init__()
        self.avgpool = GlobalAvgPooling()
        self.fc = nn.Dense(out_channels, num_classes, has_bias=True)

    def construct(self, x):
        x = self.avgpool(x)
        x = self.fc(x)
        return x

def conv7x7(in_channels, out_channels, stride=1, padding=3, has_bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=7, stride=stride, has_bias=has_bias,
                     padding=padding, pad_mode="pad")


def conv3x3(in_channels, out_channels, stride=1, padding=1, has_bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, has_bias=has_bias,
                     padding=padding, pad_mode="pad")


def conv1x1(in_channels, out_channels, stride=1, padding=0, has_bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, has_bias=has_bias,
                     padding=padding, pad_mode="pad")


class _DenseLayer(nn.Cell):
    """
    the dense layer, include 2 conv layer
    """
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.norm1 = nn.BatchNorm2d(num_input_features)
        self.relu1 = nn.ReLU()
        self.conv1 = conv1x1(num_input_features, bn_size*growth_rate)

        self.norm2 = nn.BatchNorm2d(bn_size*growth_rate)
        self.relu2 = nn.ReLU()
        self.conv2 = conv3x3(bn_size*growth_rate, growth_rate)

        # nn.Dropout in MindSpore use keep_prob, diff from Pytorch
        self.keep_prob = 1.0 - drop_rate
        self.dropout = nn.Dropout(keep_prob=self.keep_prob)

    def construct(self, features):
        bottleneck = self.conv1(self.relu1(self.norm1(features)))
        new_features = self.conv2(self.relu2(self.norm2(bottleneck)))
        if self.keep_prob < 1:
            new_features = self.dropout(new_features)
        return new_features

class _DenseBlock(nn.Cell):
    """
    the dense block
    """
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        self.cell_list = nn.CellList()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate
            )
            self.cell_list.append(layer)

        self.concate = P.Concat(axis=1)

    def construct(self, init_features):
        features = init_features
        for layer in self.cell_list:
            new_features = layer(features)
            features = self.concate((features, new_features))
        return features

### 3.2 构建Transition层

In [6]:
class _Transition(nn.Cell):
    """
    the transition layer
    """
    def __init__(self, num_input_features, num_output_features, avgpool=False):
        super(_Transition, self).__init__()
        if avgpool:
            poollayer = nn.AvgPool2d(kernel_size=2, stride=2)
        else:
            poollayer = nn.MaxPool2d(kernel_size=2, stride=2)
        self.features = nn.SequentialCell(OrderedDict([
            ('norm', nn.BatchNorm2d(num_input_features)),
            ('relu', nn.ReLU()),
            ('conv', conv1x1(num_input_features, num_output_features)),
            ('pool', poollayer)
        ]))

    def construct(self, x):
        x = self.features(x)
        return x

### 3.3 构建DenseNet主干网络框架

In [7]:
class Densenet(nn.Cell):
    """
    the densenet architecture
    """
    __constants__ = ['features']

    def __init__(self, growth_rate, block_config, num_init_features=None, bn_size=4, drop_rate=0):
        super(Densenet, self).__init__()

        layers = OrderedDict()
        if num_init_features:
            layers['conv0'] = conv7x7(3, num_init_features, stride=2, padding=3)
            layers['norm0'] = nn.BatchNorm2d(num_init_features)
            layers['relu0'] = nn.ReLU()
            layers['pool0'] = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode='same')
            num_features = num_init_features
        else:
            layers['conv0'] = conv3x3(3, growth_rate*2, stride=1, padding=1)
            layers['norm0'] = nn.BatchNorm2d(growth_rate*2)
            layers['relu0'] = nn.ReLU()
            num_features = growth_rate * 2

        # Each denseblock
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate
            )
            layers['denseblock%d'%(i+1)] = block
            num_features = num_features + num_layers*growth_rate

            if i != len(block_config)-1:
                if num_init_features:
                    trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2,
                                        avgpool=False)
                else:
                    trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2,
                                        avgpool=True)
                layers['transition%d'%(i+1)] = trans
                num_features = num_features // 2

        # Final batch norm
        layers['norm5'] = nn.BatchNorm2d(num_features)
        layers['relu5'] = nn.ReLU()

        self.features = nn.SequentialCell(layers)
        self.out_channels = num_features

    def construct(self, x):
        x = self.features(x)
        return x

    def get_out_channels(self):
        return self.out_channels

    
def _densenet121(**kwargs):
    return Densenet(growth_rate=32, block_config=(6, 12, 24, 16), num_init_features=64, **kwargs)

### 3.4 构建DenseNet121网络

In [9]:
class DenseNet121(nn.Cell):
    """
    the densenet121 architecture
    """
    def __init__(self, num_classes, include_top=True):
        super(DenseNet121, self).__init__()
        self.backbone = _densenet121()
        out_channels = self.backbone.get_out_channels()
        self.include_top = include_top
        if self.include_top:
            self.head = CommonHead(num_classes, out_channels)

        default_recurisive_init(self)
        for _, cell in self.cells_and_names():
            if isinstance(cell, nn.Conv2d):
                cell.weight.set_data(init.initializer(KaimingNormal(a=math.sqrt(5), mode='fan_out',
                                                                    nonlinearity='relu'),
                                                      cell.weight.shape,
                                                      cell.weight.dtype))
            elif isinstance(cell, nn.BatchNorm2d):
                cell.gamma.set_data(init.initializer('ones', cell.gamma.shape))
                cell.beta.set_data(init.initializer('zeros', cell.beta.shape))
            elif isinstance(cell, nn.Dense):
                cell.bias.set_data(init.initializer('zeros', cell.bias.shape))

    def construct(self, x):
        x = self.backbone(x)
        if not self.include_top:
            return x
        x = self.head(x)
        return x

In [10]:
model = DenseNet121(1000, include_top=True)
print(model)

x = Tensor(np.zeros([2, 3, 224, 224]), ms.float32)
out = model(x)

print(out.shape)

DenseNet121<
  (backbone): Densenet<
    (features): SequentialCell<
      (conv0): Conv2d<input_channels=3, output_channels=64, kernel_size=(7, 7), stride=(2, 2), pad_mode=pad, padding=3, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (norm0): BatchNorm2d<num_features=64, eps=1e-05, momentum=0.09999999999999998, gamma=Parameter (name=backbone.features.norm0.gamma, shape=(64,), dtype=Float32, requires_grad=True), beta=Parameter (name=backbone.features.norm0.beta, shape=(64,), dtype=Float32, requires_grad=True), moving_mean=Parameter (name=backbone.features.norm0.moving_mean, shape=(64,), dtype=Float32, requires_grad=False), moving_variance=Parameter (name=backbone.features.norm0.moving_variance, shape=(64,), dtype=Float32, requires_grad=False)>
      (relu0): ReLU<>
      (pool0): MaxPool2d<kernel_size=3, stride=2, pad_mode=SAME>
      (denseblock1): _DenseBlock<
        (cell_list): CellList<
          (0): _DenseLayer<
            (n

NameError: name 'Tensor' is not defined

## 4. 训练网络

In [11]:
import os
import time
import argparse
import datetime

import mindspore.nn as nn
from mindspore import Tensor
from mindspore.nn.optim import Momentum
from mindspore.communication.management import init, get_rank, get_group_size
from mindspore.train.callback import ModelCheckpoint
from mindspore.train.callback import CheckpointConfig, Callback
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
from mindspore.train.loss_scale_manager import DynamicLossScaleManager, FixedLossScaleManager
from mindspore import context
from mindspore.context import ParallelMode
from mindspore.common import set_seed
from mindspore import save_checkpoint
from mindspore.common import initializer as init


from utils.lr_scheduler import MultiStepLR, CosineAnnealingLR
from utils.crossentropy import CrossEntropy
from utils.optimizers import get_param_groups

set_seed(1)

In [12]:
class BuildTrainNetwork(nn.Cell):
    """build training network"""
    def __init__(self, network, criterion):
        super(BuildTrainNetwork, self).__init__()
        self.network = network
        self.criterion = criterion

    def construct(self, input_data, label):
        output = self.network(input_data)
        loss = self.criterion(output, label)
        return loss

### 4.1 定义回调函数

In [13]:
class ProgressMonitor(Callback):
    """monitor loss and time"""
    def __init__(self, ckpt_path="checkpoint/", ckpt_name="latest.ckpt"):
        super(ProgressMonitor, self).__init__()
        self.ckpt_path = os.path.join(ckpt_path, ckpt_name)
        
        
    def begin(self, run_context):
        print('start network train...')

    def epoch_begin(self, run_context):
        pass

    def epoch_end(self, run_context, *me_args):
        """process epoch end"""
        cb_params = run_context.original_args()
        me_step = cb_params.cur_step_num

        real_epoch = cb_params.cur_epoch_num
        print('epoch[{}], iter[{}], loss:{},'.format(real_epoch, me_step, cb_params.net_outputs))
            
        
    def step_begin(self, run_context):
        pass

    def step_end(self, run_context, *me_args):
        cb_params = run_context.original_args()
        me_step = cb_params.cur_step_num
        me_batch = cb_params.cur_step_num

        real_epoch = cb_params.cur_epoch_num
        if me_step % 100 == 0:
            print('epoch[{}], iter[{}], loss:{},'.format(real_epoch, me_step, cb_params.net_outputs))

        if me_step % 2400 == 0:
            save_checkpoint(cb_params.train_network, self.ckpt_path)
            print("saving current checkpoint at: {} ".format(self.ckpt_path))
        
    def end(self, run_context):
        cb_params = run_context.original_args()
        save_checkpoint(cb_params.train_network, self.ckpt_path)
        print("saving latest checkpoint at: {} ".format(self.ckpt_path))
        print('end network train...')



In [14]:
def train():    
    data_dir = "./dataset/train"
    de_dataset = classification_dataset(data_dir, image_size=[84, 84],
                                       per_batch_size=4, max_epoch=20,
                                       rank=0, group_size=1)

    de_dataset.map_model = 4
    epoch = 1
    steps_per_epoch = de_dataset.get_dataset_size()
    ckpt_interval = steps_per_epoch

    network = DenseNet121(100)

    criterion = CrossEntropy(smooth_factor=0.0, num_classes=100)

    lr_scheduler = CosineAnnealingLR(lr=0.1,
                                     T_max=120,
                                     steps_per_epoch=steps_per_epoch,
                                     max_epoch=epoch,
                                     warmup_epochs=0,
                                     eta_min=0)
    lr_schedule = lr_scheduler.get_lr()

    opt = Momentum(params=get_param_groups(network),
                       learning_rate=Tensor(lr_schedule),
                       momentum=0.9,
                       weight_decay=0.0001,
                       loss_scale=1024)

    criterion.add_flags_recursive(fp32=True)


    train_net = BuildTrainNetwork(network, criterion)
    loss_scale_manager = FixedLossScaleManager(1024, drop_overflow_update=False)
    model = Model(train_net, optimizer=opt, metrics=None, loss_scale_manager=loss_scale_manager, amp_level="O3")

    progress_cb = ProgressMonitor(ckpt_path="checkpoint/", ckpt_name="cur.ckpt")
    callbacks = [progress_cb,]

    '''
    outputs_dir = os.path.join("checkpoint/",datetime.datetime.now().strftime('%Y-%m-%d_time_%H_%M_%S'))
    ckpt_max_num = epoch * steps_per_epoch // ckpt_interval
    ckpt_config = CheckpointConfig(save_checkpoint_steps=ckpt_interval,
                                           keep_checkpoint_max=ckpt_max_num)
    ckpt_cb = ModelCheckpoint(config=ckpt_config, directory=outputs_dir)
    callbacks.append(ckpt_cb)
    '''

    model.train(epoch, de_dataset, callbacks=callbacks)
    
train()

[WARNING] ME(13820:9872,MainProcess):2022-09-22-19:21:38.380.769 [mindspore\nn\loss\loss.py:161] '_Loss' is deprecated from version 1.3 and will be removed in a future version, use 'LossBase' instead.
[WARNING] ME(13820:9872,MainProcess):2022-09-22-19:21:38.529.084 [mindspore\train\model.py:536] The CPU cannot support dataset sink mode currently.So the training process will be performed with dataset not sink.


start network train...


KeyboardInterrupt: 

## 5. 模型效果评估

In [15]:
import argparse
import glob

from mindspore.communication.management import init, get_rank, get_group_size, release
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.ops import operations as P
from mindspore.ops import functional as F
from mindspore.common import dtype as mstype
from mindspore.common import initializer as init
from mindspore import Tensor

In [16]:
per_batch_size = 4

def get_top5_acc(top5_arg, gt_class):
    sub_count = 0
    for top5, gt in zip(top5_arg, gt_class):
        if gt in top5:
            sub_count += 1
    return sub_count

def evaluation():
    data_dir = "./dataset/val"
    de_dataset = classification_dataset(data_dir, image_size=[84, 84],
                                       per_batch_size=4, max_epoch=1,
                                       rank=0, group_size=1,mode='eval')
    eval_dataloader = de_dataset.create_tuple_iterator()
    
    network = DenseNet121(100)
    load_checkpoint("checkpoint/cur.ckpt", net=network)
    
    network.add_flags_recursive(fp16=True)
    
    img_tot = 0
    top1_correct = 0
    top5_correct = 0
    network.set_train(False)
    for data, gt_classes in eval_dataloader:
        output = network(Tensor(data, mstype.float32))
        output = output.asnumpy()
        gt_classes = gt_classes.asnumpy()

        top1_output = np.argmax(output, (-1))
        top5_output = np.argsort(output)[:, -5:]

        t1_correct = np.equal(top1_output, gt_classes).sum()
        top1_correct += t1_correct
        top5_correct += get_top5_acc(top5_output, gt_classes)
        img_tot += per_batch_size
        
    results = [[top1_correct], [top5_correct], [img_tot]]
    results = np.array(results)
    
    print('results={}'.format(results))
    
    top1_correct = results[0, 0]
    top5_correct = results[1, 0]
    img_tot = results[2, 0]
    acc1 = 100.0 * top1_correct / img_tot
    acc5 = 100.0 * top5_correct / img_tot
    print('top1_correct={}, tot={}, acc={:.2f}%'.format(top1_correct,img_tot,acc1))
    print('top5_correct={}, tot={}, acc={:.2f}%'.format(top5_correct,img_tot,acc5))
    
    
evaluation()

[WARNING] ME(13820:9872,MainProcess):2022-09-22-19:22:21.823.623 [mindspore\train\serialization.py:667] For 'load_param_into_net', remove parameter network.backbone.features.conv0.weight's prefix name: network., continue to load it to net parameter backbone.features.conv0.weight.


results=[[  237]
 [ 1037]
 [12000]]
top1_correct=237, tot=12000, acc=1.98%
top5_correct=1037, tot=12000, acc=8.64%


## 6. 模型预测

In [203]:
import pandas as pd

# 根据分类索引列表获取分类名称
def class_index_map(df, predict_output, index_dict):
    result_list = []
    for i in predict_output:
        temp_df = df.loc[:,df.loc[0,:] == index_dict[i]] # 获取分类编号对应的df所有行，index_dict[i]为分类编号，如“n012345”
        class_name = str(temp_df.loc[1,:].values) # 获取分类编号对应的分类名称
        result_list.append(class_name)
        
    return result_list

def prediction(classes_df, index_dict):
    data_dir = "./dataset/test"
    de_dataset = classification_dataset(data_dir, image_size=[84, 84],
                                       per_batch_size=4, max_epoch=1,
                                       rank=0, group_size=1,mode='test')
    eval_dataloader = de_dataset.create_tuple_iterator()
    
    network = DenseNet121(100)
    load_checkpoint("checkpoint/cur.ckpt", net=network)
    
    network.add_flags_recursive(fp16=True)
    
    network.set_train(False)
    i = 0
    img_tot = 0
    for data, gt_classes in eval_dataloader:
        output = network(Tensor(data, mstype.float32))
        output = output.asnumpy()
        gt_classes = gt_classes.asnumpy()

        top1_output = np.argmax(output, (-1))
        top1_output = class_index_map(classes_df, top1_output, index_dict)
        gt_classes = class_index_map(classes_df, gt_classes, index_dict)
        print("predict classification: \n{}".format(top1_output))
        print("real classification: \n{}".format(gt_classes))
        print("--------------------------No.{} batch of test dataset----------------------------".format(i+1))
        i += 1


# 存储分类编号和分类名称，如"n012345==>Cat"
classes_df = pd.read_json('class_dict.json')
# 存储分类索引和分类编号，如“0==>n012345”
index_dict = {}
data_dir = "dataset/train"
index_num = os.listdir(data_dir)
index_num.sort()
i = 0
for index in index_num:
    index_dict[i] = index
    i += 1        
               
prediction(classes_df, index_dict)

[WARNING] ME(9133:8606791168,MainProcess):2022-09-15-17:14:53.701.172 [mindspore/dataset/core/validator_helpers.py:804] 'Decode' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Decode' from mindspore.dataset.vision instead.
[WARNING] ME(9133:8606791168,MainProcess):2022-09-15-17:14:53.701.742 [mindspore/dataset/core/validator_helpers.py:804] 'Resize' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Resize' from mindspore.dataset.vision instead.
[WARNING] ME(9133:8606791168,MainProcess):2022-09-15-17:14:53.702.034 [mindspore/dataset/core/validator_helpers.py:804] 'CenterCrop' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'CenterCrop' from mindspore.dataset.vision instead.
[WARNING] ME(9133:8606791168,MainProcess):2022-09-15-17:14:53.702.277 [mindspore/dataset/core/validator_

predict classification: 
["['tank']", "['catamaran']", "['catamaran']", "['photocopier']"]
real classification: 
["['robin']", "['catamaran']", "['jellyfish']", "['street_sign']"]
--------------------------No.1 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['mixing_bowl']", "['white_wolf']", "['carton']", "['robin']"]
--------------------------No.2 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['tank']", "['catamaran']", "['triceratops']"]
real classification: 
["['aircraft_carrier']", "['missile']", "['street_sign']", "['meerkat']"]
--------------------------No.3 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['tank']", "['catamaran']", "['triceratops']"]
real classification: 
["['solar_dish']", "['clog']", "['chime']", "['white_wolf']"]
--------------------------No.4 batch of test dataset--

predict classification: 
["['ear']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['ladybug']", "['carton']", "['French_bulldog']", "['tile_roof']"]
--------------------------No.33 batch of test dataset----------------------------
predict classification: 
["['tank']", "['catamaran']", "['tank']", "['tank']"]
real classification: 
["['oboe']", "['dishrag']", "['unicycle']", "['Newfoundland']"]
--------------------------No.34 batch of test dataset----------------------------
predict classification: 
["['triceratops']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['meerkat']", "['prayer_rug']", "['vase']", "['iPod']"]
--------------------------No.35 batch of test dataset----------------------------
predict classification: 
["['photocopier']", "['tank']", "['tank']", "['ear']"]
real classification: 
["['vase']", "['robin']", "['hair_slide']", "['boxer']"]
--------------------------No.36 batch of test dataset----------------------------
predict classification:

predict classification: 
["['tank']", "['tank']", "['triceratops']", "['catamaran']"]
real classification: 
["['tank']", "['scoreboard']", "['worm_fence']", "['missile']"]
--------------------------No.67 batch of test dataset----------------------------
predict classification: 
["['consomme']", "['tank']", "['tank']", "['ear']"]
real classification: 
["['golden_retriever']", "['African_hunting_dog']", "['carousel']", "['trifle']"]
--------------------------No.68 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['photocopier']", "['tank']"]
real classification: 
["['clog']", "['catamaran']", "['malamute']", "['bookshop']"]
--------------------------No.69 batch of test dataset----------------------------
predict classification: 
["['triceratops']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['bolete']", "['organ']", "['black-footed_ferret']", "['ant']"]
--------------------------No.70 batch of test dataset--------------

predict classification: 
["['photocopier']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['rock_beauty']", "['triceratops']", "['cannon']", "['Newfoundland']"]
--------------------------No.101 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['goose']", "['white_wolf']", "['parallel_bars']", "['vase']"]
--------------------------No.102 batch of test dataset----------------------------
predict classification: 
["['photocopier']", "['catamaran']", "['tank']", "['tank']"]
real classification: 
["['hourglass']", "['coral_reef']", "['file']", "['frying_pan']"]
--------------------------No.103 batch of test dataset----------------------------
predict classification: 
["['triceratops']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['rock_beauty']", "['cocktail_shaker']", "['unicycle']", "['lipstick']"]
--------------------------No.104 batch of test dataset--------

...

predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['miniature_poodle']", "['aircraft_carrier']", "['bookshop']", "['upright']"]
--------------------------No.2850 batch of test dataset----------------------------
predict classification: 
["['photocopier']", "['tank']", "['catamaran']", "['tank']"]
real classification: 
["['white_wolf']", "['yawl']", "['miniskirt']", "['boxer']"]
--------------------------No.2851 batch of test dataset----------------------------
predict classification: 
["['ear']", "['green_mamba']", "['catamaran']", "['catamaran']"]
real classification: 
["['carton']", "['hotdog']", "['parallel_bars']", "['tile_roof']"]
--------------------------No.2852 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['photocopier']"]
real classification: 
["['bolete']", "['vase']", "['reel']", "['cannon']"]
--------------------------No.2853 batch of test dataset-------------

predict classification: 
["['tank']", "['photocopier']", "['catamaran']", "['tank']"]
real classification: 
["['miniskirt']", "['hourglass']", "['catamaran']", "['house_finch']"]
--------------------------No.2884 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['Arctic_fox']", "['poncho']", "['solar_dish']", "['fire_screen']"]
--------------------------No.2885 batch of test dataset----------------------------
predict classification: 
["['tank']", "['catamaran']", "['tank']", "['tank']"]
real classification: 
["['trifle']", "['yawl']", "['Tibetan_mastiff']", "['dalmatian']"]
--------------------------No.2886 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['green_mamba']", "['ear']", "['tank']"]
real classification: 
["['dugong']", "['Saluki']", "['pencil_box']", "['organ']"]
--------------------------No.2887 batch of test dataset-------------

predict classification: 
["['tank']", "['catamaran']", "['tank']", "['tank']"]
real classification: 
["['ladybug']", "['solar_dish']", "['carousel']", "['combination_lock']"]
--------------------------No.2918 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['photocopier']"]
real classification: 
["['beer_bottle']", "['slot']", "['trifle']", "['vase']"]
--------------------------No.2919 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['tank']", "['triceratops']", "['tank']"]
real classification: 
["['yawl']", "['upright']", "['oboe']", "['iPod']"]
--------------------------No.2920 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['tank']", "['catamaran']", "['tank']"]
real classification: 
["['Walker_hound']", "['upright']", "['chime']", "['Walker_hound']"]
--------------------------No.2921 batch of test dataset----------------------

predict classification: 
["['catamaran']", "['tank']", "['consomme']", "['ear']"]
real classification: 
["['goose']", "['ladybug']", "['orange']", "['toucan']"]
--------------------------No.2952 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['rhinoceros_beetle']", "['stage']", "['bolete']", "['harvestman']"]
--------------------------No.2953 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['ear']", "['tank']"]
real classification: 
["['scoreboard']", "['ladybug']", "['chime']", "['Walker_hound']"]
--------------------------No.2954 batch of test dataset----------------------------
predict classification: 
["['tank']", "['photocopier']", "['tank']", "['tank']"]
real classification: 
["['cannon']", "['mixing_bowl']", "['cocktail_shaker']", "['harvestman']"]
--------------------------No.2955 batch of test dataset-------------------------

predict classification: 
["['tank']", "['tank']", "['tank']", "['photocopier']"]
real classification: 
["['scoreboard']", "['crate']", "['meerkat']", "['harvestman']"]
--------------------------No.2986 batch of test dataset----------------------------
predict classification: 
["['tank']", "['photocopier']", "['photocopier']", "['triceratops']"]
real classification: 
["['white_wolf']", "['iPod']", "['malamute']", "['theater_curtain']"]
--------------------------No.2987 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['combination_lock']", "['bookshop']", "['house_finch']", "['prayer_rug']"]
--------------------------No.2988 batch of test dataset----------------------------
predict classification: 
["['tank']", "['photocopier']", "['tank']", "['catamaran']"]
real classification: 
["['miniature_poodle']", "['snorkel']", "['upright']", "['coral_reef']"]
--------------------------No.2989 batc

In [176]:
index_dict

{0: 'n02795169',
 1: 'n03417042',
 2: 'n04251144',
 3: 'n03476684',
 4: 'n01910747',
 5: 'n04509417',
 6: 'n03838899',
 7: 'n09256479',
 8: 'n03062245',
 9: 'n03207743',
 10: 'n07697537',
 11: 'n02138441',
 12: 'n03527444',
 13: 'n04612504',
 14: 'n01981276',
 15: 'n02950826',
 16: '.DS_Store',
 17: 'n01770081',
 18: 'n03854065',
 19: 'n03337140',
 20: 'n02111277',
 21: 'n02113712',
 22: 'n03017168',
 23: 'n13133613',
 24: 'n03146219',
 25: 'n04418357',
 26: 'n02108551',
 27: 'n03544143',
 28: 'n03998194',
 29: 'n02099601',
 30: 'n02074367',
 31: 'n01749939',
 32: 'n02981792',
 33: 'n02823428',
 34: 'n02114548',
 35: 'n03676483',
 36: 'n01558993',
 37: 'n03272010',
 38: 'n02606052',
 39: 'n02108915',
 40: 'n04258138',
 41: 'n03775546',
 42: 'n03888605',
 43: 'n04243546',
 44: 'n02091244',
 45: 'n02219486',
 46: 'n01855672',
 47: 'n09246464',
 48: 'n06794110',
 49: 'n02165456',
 50: 'n03127925',
 51: 'n03584254',
 52: 'n13054560',
 53: 'n02116738',
 54: 'n03773504',
 55: 'n02105505',
 5

In [113]:
import pandas as pd

df = pd.read_json('class_dict.json')


In [115]:
df.loc[0,:]=="n01484850"

0      False
1      False
2       True
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: 0, Length: 1000, dtype: bool

In [206]:
a = df.loc[:,df.loc[0,:]=="n01484850"]
a

,2
0,n01484850
1,great_white_shark


In [173]:
str(a.loc[1,:].values)

"['great_white_shark']"

In [89]:
"n01440764" in df.loc[0, :].values

True

In [124]:
top1_output = [34, 54, 51, 5]
index_dict = {5:"n02145123", 51:"n01484850", 2:"n02145123"}

In [132]:
index_dict = {}
data_dir = "dataset/train"
index_num = os.listdir(data_dir)
i = 0
for index in index_num:
    index_dict[i] = index
    i += 1
    
def index_num_map(top1_output):
    result_list = []
    for i in top1_output:
        temp_df = df.loc[:,df.loc[0,:]== index_dict[i]]
        class_name = temp_df.loc[1,:].values
        result_list.append(class_name)
        
    return result_list

In [133]:
index_num_map(top1_output)

[array(['white_wolf'], dtype=object),
 array(['missile'], dtype=object),
 array(['iPod'], dtype=object),
 array(['unicycle'], dtype=object)]